In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlalchemy
import os

# 1. Connexion Base de données
db_path = os.path.abspath('../data/warehouse/northwind_dwh.db')
engine = sqlalchemy.create_engine(f'sqlite:///{db_path}')
print(f"✅ Connexion : {db_path}")

# ==============================================================================
# ÉTAPE CLÉ : DÉTECTION AUTOMATIQUE DES COLONNES
# ==============================================================================
def get_valid_columns(table_name, engine):
    try:
        # On lit juste les entêtes (LIMIT 0)
        return pd.read_sql(f"SELECT * FROM {table_name} LIMIT 0", engine).columns.tolist()
    except Exception as e:
        print(f"⚠️ Impossible de lire {table_name}: {e}")
        return []

# Récupérer les colonnes réelles de DimClient
client_cols = get_valid_columns('DimClient', engine)
print(f"🔍 Colonnes trouvées dans DimClient : {client_cols}")

# Déterminer quelle colonne utiliser pour la Géographie (Country, Pays, ou City...)
geo_column = None
possible_geo_names = ['Country', 'Pays', 'Region', 'City', 'Ville']

for col in possible_geo_names:
    if col in client_cols:
        geo_column = col
        break

print(f"🌍 Colonne géographique choisie : {geo_column if geo_column else 'AUCUNE (Graphiques géo désactivés)'}")

# ==============================================================================
# CONSTRUCTION DE LA REQUÊTE SQL DYNAMIQUE
# ==============================================================================
# On ne sélectionne que ce qui existe
sql_geo_part = f", c.{geo_column}" if geo_column else ""

query = f"""
SELECT 
    f.TotalAmount, 
    f.Quantity,
    d.Year, 
    d.MonthName, 
    e.FullName as Employee
    {sql_geo_part}
FROM FactSales f
LEFT JOIN DimDate d ON f.DateKey = d.DateKey
LEFT JOIN DimClient c ON f.CustomerKey = c.CustomerKey
LEFT JOIN DimEmployee e ON f.EmployeeKey = e.EmployeeKey
"""

try:
    df = pd.read_sql(query, engine)
    print("✅ Données chargées avec succès !")
    display(df.head())
except Exception as e:
    print(f"❌ Erreur SQL : {e}")
    # Arrêt du script si la requête échoue
    raise e

# ==============================================================================
# VISUALISATIONS (KPI & GRAPHIQUES)
# ==============================================================================

# 1. KPI
print("\n--- KPI ---")
print(f"💰 Total Ventes : {df['TotalAmount'].sum():,.2f} $")
print(f"📦 Nombre Commandes (lignes) : {len(df)}")

# 2. Graphique par Employé
print("\n--- Performance Employés ---")
sales_emp = df.groupby('Employee')['TotalAmount'].sum().reset_index()
fig_emp = px.bar(sales_emp, x='Employee', y='TotalAmount', title="Ventes par Employé")
fig_emp.show()

# 3. Graphique 3D & Carte (Seulement si une colonne géo a été trouvée)
if geo_column:
    print(f"\n--- Analyse Géographique (basée sur '{geo_column}') ---")
    
    # Carte
    sales_geo = df.groupby(geo_column)['TotalAmount'].sum().reset_index()
    fig_map = px.choropleth(sales_geo, 
                            locations=geo_column, 
                            locationmode='country names', # Fonctionne mieux si c'est des pays
                            color="TotalAmount", 
                            title=f"Carte des Ventes ({geo_column})")
    fig_map.show()
    
    # 3D
    df_3d = df.groupby(['Year', geo_column, 'Employee']).agg({'TotalAmount': 'sum'}).reset_index()
    fig_3d = px.scatter_3d(df_3d, 
                           x='Year', y=geo_column, z='Employee',
                           size='TotalAmount', color='TotalAmount',
                           title=f"Cube OLAP 3D (Année x {geo_column} x Employé)")
    fig_3d.show()
else:
    print("\n⚠️ Pas de colonne géographique détectée (Country/Pays/City). La carte et le graphe 3D sont masqués.")
    print("💡 Conseil : Vérifiez votre fichier 'Customers.csv' dans data/raw/.")

# 4. Statut Livraison (Depuis RAW car souvent absent du DWH)
try:
    raw_path = '../data/raw/Orders.csv'
    if os.path.exists(raw_path):
        raw_orders = pd.read_csv(raw_path)
        # Gestion des colonnes dates mal nommées
        date_col = 'ShippedDate' if 'ShippedDate' in raw_orders.columns else 'Shipped Date'
        
        if date_col in raw_orders.columns:
            raw_orders['Status'] = raw_orders[date_col].apply(lambda x: 'Non Livrée' if pd.isnull(x) else 'Livrée')
            fig_pie = px.pie(raw_orders, names='Status', title='Statut des Livraisons')
            fig_pie.show()
except Exception as e:
    print(f"Info: Pas de graphe livraison ({e})")